In [1]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 24: Air Duct Spelunking ---</h2><p>You've finally met your match; the doors that provide access to the roof are locked tight, and all of the controls and related electronics are inaccessible. You simply can't reach them.</p>
<p>The robot that cleans the air ducts, however, <em>can</em>.</p>
<p>It's not a very fast <span title="The Brave Little Air Duct Cleaning Robot That Could">little robot</span>, but you reconfigure it to be able to interface with some of the exposed wires that have been routed through the <a href="https://en.wikipedia.org/wiki/HVAC">HVAC</a> system. If you can direct it to each of those locations, you should be able to bypass the security controls.</p>
<p>You extract the duct layout for this area from some blueprints you acquired and create a map with the relevant locations marked (your puzzle input). <code>0</code> is your current location, from which the cleaning robot embarks; the other numbers are (in <em>no particular order</em>) the locations the robot needs to visit at least once each. Walls are marked as <code>#</code>, and open passages are marked as <code>.</code>. Numbers behave like open passages.</p>
<p>For example, suppose you have a map like the following:</p>
<pre><code>###########
#0.1.....2#
#.#######.#
#4.......3#
###########
</code></pre>
<p>To reach all of the points of interest as quickly as possible, you would have the robot take the following path:</p>
<ul>
<li><code>0</code> to <code>4</code> (<code>2</code> steps)</li>
<li><code>4</code> to <code>1</code> (<code>4</code> steps; it can't move diagonally)</li>
<li><code>1</code> to <code>2</code> (<code>6</code> steps)</li>
<li><code>2</code> to <code>3</code> (<code>2</code> steps)</li>
</ul>
<p>Since the robot isn't very fast, you need to find it the <em>shortest route</em>. This path is the fewest steps (in the above example, a total of <code>14</code>) required to start at <code>0</code> and then visit every other location at least once.</p>
<p>Given your actual map, and starting from location <code>0</code>, what is the <em>fewest number of steps</em> required to visit every non-<code>0</code> number marked on the map at least once?</p>
</article>


In [2]:
from collections import deque
from curses.ascii import isdigit
from tabulate import tabulate


example_I = """
###########
#0.1.....2#
#.#######.#
#4.......3#
###########
"""
# [
#     (1, 1), (2, 1), (3, 1), (2, 1), (1, 1),
#     (1, 2), (1, 3), (1, 4), (1, 5), (1, 6),
#     (1, 7), (1, 8), (1, 9), (2, 9),
# ]


example_II = """
###########
#0.1.....2#
#.#########
#4.......3#
###########
"""
# [
#     (1, 1), (1, 2), (1, 3), (1, 4), (1, 5),
#     (1, 6), (1, 7), (1, 8), (1, 9), (1, 8),
#     (1, 7), (1, 6), (1, 5), (1, 4), (1, 3),
#     (1, 2), (1, 1), (2, 1), (3, 1), (3, 2),
#     (3, 3), (3, 4), (3, 5), (3, 6), (3, 7),
#     (3, 8),
# ]


def minimal_steps(grst: str, go_back: bool = False) -> int:
    grid = [list(line) for line in grst.strip().splitlines()]
    rows, cols = len(grid), len(grid[0])
    numbers = {
        grid[r][c]: (r, c)
        for r, c in product(range(rows), range(cols))
        if grid[r][c].isdigit()
    }
    visited = set()

    queue = deque([(numbers["0"], list(numbers.keys()), [])])

    while queue:
        (r, c), notseen, path = queue.popleft()

        if grid[r][c].isdigit() and grid[r][c] in notseen:
            notseen.remove(grid[r][c])

        if not notseen and (not go_back or (r, c) == numbers["0"]):
            return len(path)

        to_be_hashed = (r, c, tuple(notseen), path[-1] if path else None)
        if to_be_hashed in visited:
            continue

        visited.add(to_be_hashed)

        taken = False
        for d_r, d_c in ((-1, 0), (0, 1), (1, 0), (0, -1)):
            if (
                0 <= r + d_r < rows
                and 0 <= c + d_c < cols
                and grid[r + d_r][c + d_c] != "#"
                and (not path or grid[r][c].isdigit() or (r + d_r, c + d_c) != path[-1])
            ):
                taken = True
                queue.append(
                    (
                        (r + d_r, c + d_c),
                        list(notseen),
                        path + [(r, c)],
                    )
                )

        if not taken:
            queue.append(
                (
                    path[-1],
                    list(notseen),
                    path + [(r, c)],
                )
            )


print("example_I : ", minimal_steps(example_I))
print("example_II: ", minimal_steps(example_II))

example_I :  14
example_II:  26


In [3]:
with open("../input/day24.txt") as f:
    part_I = f.read()

minimal_steps(part_I)

474

<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>474</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Of course, if you leave the cleaning robot somewhere weird, someone is bound to notice.</p>
<p>What is the fewest number of steps required to start at <code>0</code>, visit every non-<code>0</code> number marked on the map at least once, and then <em>return to <code>0</code></em>?</p>
</article>

</main>


In [4]:
print("example_I : ", minimal_steps(example_I, True))
example_I = """
###########
#0.1.....2#
#.#######.#
#4.......3#
###########
"""
# [
#     (1, 1), (1, 2), (1, 3), (1, 4), (1, 5),
#     (1, 6), (1, 7), (1, 8), (1, 9), (2, 9),
#     (3, 9), (3, 8), (3, 7), (3, 6), (3, 5),
#     (3, 4), (3, 3), (3, 2), (3, 1), (2, 1),
# ]


print("example_II: ", minimal_steps(example_II, True))
example_II = """
###########
#0.1.....2#
#.#########
#4.......3#
###########
"""
# [
#     (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6),
#     (1, 7), (1, 8), (1, 9), (1, 8), (1, 7), (1, 6),
#     (1, 5), (1, 4), (1, 3), (1, 2), (1, 1), (2, 1),
#     (3, 1), (3, 2), (3, 3), (3, 4), (3, 5), (3, 6),
#     (3, 7), (3, 8), (3, 9), (3, 8), (3, 7), (3, 6),
#     (3, 5), (3, 4), (3, 3), (3, 2), (3, 1), (2, 1)
# ]

example_I :  20
example_II:  36


In [5]:
with open("../input/day24.txt") as f:
    part_I = f.read()

minimal_steps(part_I, True)

696

<link href="style.css" rel="stylesheet"></link>

<p>Your puzzle answer was <code>696</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
